In [ ]:
import pathlib
import polars as pl

datadir = pathlib.Path.cwd().parent / "data"

pl_aozora = pl.read_parquet(datadir / "aozora.parquet")

# 13章 文字列
## 13-1 部分文字列の検出・抽出・置換
### Q: URLから芥川竜之介の作品を判定して抽出


#### Awesome

In [ ]:
pl_aozora.filter(pl.col("url").str.contains("/000879/"))

### Q: 出版年月から年と月を位置指定で抽出


#### Awesome

In [ ]:
pl_aozora.select([
    "text_id",
    "title",
    "published_at",
    # published_atの文字列の一部を抽出
    pl.col("published_at").str.slice(0, 4).alias("year"),
    pl.col("published_at").str.slice(5, 2).alias("month")
])

### Q: 出版年月のフォーマットを`YYYY年MM月`から`YYYY-MM`に変換（部分文字列の検出、置換）


#### Awesome

In [ ]:
pl_aozora.select([
    "text_id",
    "title",
    # published_atが"月"を含む場合は"年"を"-"に置換して"月"を削除、それ以外は"年"を削除
    pl.when(pl.col("published_at").str.contains("月"))
        .then(pl.col("published_at").str.replace("年", "-").str.replace("月", ""))
        .otherwise(pl.col("published_at").str.replace("年", ""))
])

### Q: URLから人物IDと作品IDを抽出 (正規表現による抽出)


#### Awesome

In [ ]:
pl_aozora.select([
    "title",
    "url",
    # urlから正規表現で抽出
    pl.col("url").str.extract(r"cards/0*([1-9][0-9]*)/").alias("author_id"),
    pl.col("url").str.extract(r"card([0-9]+)\.").alias("text_id")
])

## 13-2 区切り文字による文字列分割
### Q: 句点で文に分割


#### Awesome

In [ ]:
pl_aozora.select([
    "text_id",
    "title",
    # "。"でtextを分割
    pl.col("text").str.split("。").alias("sentence_list")
])

### Q: 複数の区切り文字で文に分割


#### Awesome

In [ ]:
pl_aozora.select([
    "text_id",
    "title",
    # "\n"を"。"に置換した上で、"。"でtextを分割
    pl.col("text").str.replace_all("\n", "。").str.split("。").alias("sentence_list")
])

### Q: 分割結果の配列を列に展開


#### Awesome

In [ ]:
(
    pl_aozora
    #（1）"。"でtextを分割
    .with_columns(sentence_list=pl.col("text").str.split("。"))
    .select([
        "text_id",
        "title",
        #（2）sentence_listの0番目と1番目の要素を取得
        pl.col("sentence_list").list.get(0).alias("first_sentence"),
        pl.col("sentence_list").list.get(1).alias("second_sentence")
    ])
)

### Q: 分割結果の配列を行に展開


#### Awesome

In [ ]:
(
    pl_aozora
    .select([
        "text_id",
        "title",
        #（1）"。"でtextを分割
        pl.col("text").str.split("。").alias("sentence")
    ])
    #（2）sentenceを複数行に展開
    .explode("sentence")
)

## 13-3 文字列データのクレンジング
### Q: ユニコード正規化


#### Awesome

In [ ]:
import unicodedata
pl_aozora.select([
    "text_id",
    "title",
    # textをユニコード正規化
    pl.col("text").map_elements(lambda x: unicodedata.normalize("NFKC", x))
])

### Q: 不要な文字列パターンの除去（正規表現による除去）


#### Awesome

In [ ]:
import unicodedata
pl_aozora_after_cleansing = (
    pl_aozora
    # （1） 文への分割と行展開
    .select([
        "text_id",
        "title",
        pl.col("text").str.replace_all("\n", "。").str.split("。").alias("sentence")
    ])
    .explode("sentence")
    .with_columns(
        sentence=
            pl.col("sentence")
            # （2） ユニコード正規化
            .map_elements(lambda x: unicodedata.normalize("NFKC", x))
            # （3） 正規表現による不要パターンの除去
            .str.replace_all(r"《.*?》|\[#.*?\]|[\|※「」()]", "")
            # （4） 先頭や末尾のスペースの除去
            .str.strip_chars()
    )
    # （5） 長さが1文字以上の行のみ抽出
    .filter(pl.col("sentence").str.len_bytes() >= 1)
)
pl_aozora_after_cleansing

## 13-4 形態素解析による日本語文章の単語分解と単語抽出
### Q: 文を単語に分解（形態素解析）


#### Awesome

In [ ]:
# spaCyおよびGiNZAモジュールの読み込み
# 初回はpipでインストールする
# pip install spacy ja-ginza
import spacy
nlp = spacy.load("ja_ginza")

pl_aozora_token = (
    pl_aozora_after_cleansing
    # （1） 単語（Tokenオブジェクト）への分解と行展開
    .pipe(lambda df: df.with_columns(token=
        pl.Series([list(d) for d in nlp.pipe(df["sentence"])])
    ))
    .select(["text_id", "title", "token"])
    .explode("token")
    # （2） Tokenオブジェクトのプロパティから列を作成
    .select([
        "text_id",
        "title",
        # 元の文章での表記
        pl.col("token").map_elements(lambda e: e.text).alias("text"),
        # 単語の原型
        pl.col("token").map_elements(lambda e: e.lemma_).alias("lemma"),
        # 単語の品詞
        pl.col("token").map_elements(lambda e: e.pos_).alias("pos"),
        # 単語がストップワードかどうかを表すブール値
        pl.col("token").map_elements(lambda e: e.is_stop).alias("is_stop"),
        # 単語にベクトル表現が付与されているかどうかを表すブール値
        pl.col("token").map_elements(lambda e: e.has_vector).alias("has_vector"),
        # 単語のベクトル表現（付与されていない場合はゼロベクトル）
        pl.col("token").map_elements(lambda e: e.vector).alias("vector"),
    ])
)
pl_aozora_token

### Q: 特定の品詞の抽出とストップワードの除去


#### Awesome

In [ ]:
# 名詞と固有名詞の抽出とストップワードの除去
pl_aozora_token.filter(
    pl.col("pos").is_in(["NOUN", "PROPN"]) & pl.col("is_stop").not_()
)

## 13-5 文章のベクトル化
### Q: bag-of-wordsベクトルの作成


#### Awesome

In [ ]:
(
    pl_aozora_token
    # （1） 名詞と固有名詞の抽出とストップワードの除去
    .filter(pl.col("pos").is_in(["NOUN", "PROPN"]) & pl.col("is_stop").not_())
    # （2） text_idごとにlemmaの値をカウントし、横持ち行列に変換
    .pivot(values="text", index="text_id", columns="lemma",
        aggregate_function="len", sort_columns=True)
)

### Q: TF-IDF 特徴量の作成


#### Awesome

In [ ]:
import numpy as np

(
    pl_aozora_token
    # （1） 名詞と固有名詞の抽出とストップワードの除去
    .filter(pl.col("pos").is_in(["NOUN", "PROPN"]) & pl.col("is_stop").not_())
    # （2） text_idごとにlemmaの値をカウント
    .group_by(["text_id", "lemma"]).len()
    # （3） TF-IDFを計算
    # （3）-1 TF=ある文章における対象単語の出現回数/ある文章に含まれる全単語の出現回数の合計
    # （3）-2 IDF=log(全文章数/対象単語が含まれる文章数) + 1
    .with_columns(
        tf=pl.col("len") / pl.col("len").sum().over("text_id"),
        idf=(pl.col("text_id").n_unique()
            / pl.col("text_id").n_unique().over("lemma"))
            .log() + 1
    )
    # （3）-3 TF-IDF = TF * IDF
    .with_columns(tf_idf=pl.col("tf")*pl.col("idf"))
    # （3）-4 TF-IDFを文章ごとに計算したL2ノルムで割って正規化
    .with_columns(tf_idf_normalized=
        pl.col("tf_idf") / pl.col("tf_idf").map_elements(np.linalg.norm).over("text_id"))
    # （4） 横持ち変換
    .pivot(index="text_id", columns="lemma", values="tf_idf_normalized",
        aggregate_function="first", sort_columns=True)
)

### Q: 平均単語ベクトル特徴量の作成


#### Awesome

In [ ]:
(
    pl_aozora_token
    # （1） 名詞と固有名詞の抽出とストップワードの除去
    .filter(pl.col("pos").is_in(["NOUN", "PROPN"]) & pl.col("is_stop").not_())
    # （2） 作品ごとに単語ベクトルの平均を計算
    .group_by("text_id").map_groups(lambda df:
        pl.DataFrame(df["vector"].to_numpy().mean().reshape(1, -1))
        .with_columns(text_id=df["text_id"][0])
        )
)